<a href="https://colab.research.google.com/github/danielhou13/cogs402longformer/blob/main/src/CaptumLongformerSequenceClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook adapts the Captum tutorial for question answering and refactors it into the longformer sequence classification task. Specifically, this notebook focuses on using the model's embeddings to get word attributions for the examples of your choice, or the entire dataset if needed.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Import dependencies

In [1]:
import sys
sys.path.append('/content/drive/My Drive/{}'.format("cogs402longformer/"))

In [2]:
pip install transformers --quiet

In [3]:
pip install captum --quiet

In [3]:
pip install datasets --quiet

In [4]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [5]:
from captum.attr import visualization as viz
from captum.attr import IntegratedGradients, LayerConductance, LayerIntegratedGradients
from captum.attr import configure_interpretable_embedding_layer, remove_interpretable_embedding_layer

import torch
import pandas as pd

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Import model

In [7]:
from transformers import LongformerForSequenceClassification, LongformerTokenizer, LongformerConfig
# replace <PATH-TO-SAVED-MODEL> with the real path of the saved model
model_path = 'danielhou13/longformer-finetuned_papers_v2'
#model_path = 'danielhou13/longformer-finetuned-new-cogs402'

# load model
model = LongformerForSequenceClassification.from_pretrained(model_path, num_labels = 2)
model.to(device)
model.eval()
model.zero_grad()

# load tokenizer
tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")

Create functions that give us the input ids and the position ids for the text we want to examine

In [8]:
def predict(inputs, position_ids=None, attention_mask=None):
    output = model(inputs,
                   position_ids=position_ids,
                   attention_mask=attention_mask)
    return output.logits

In [9]:
ref_token_id = tokenizer.pad_token_id # A token used for generating token reference
sep_token_id = tokenizer.sep_token_id # A token used as a separator between question and text and it is also added to the end of the text.
cls_token_id = tokenizer.cls_token_id # A token used for prepending to the concatenated question-text word sequence

In [10]:
max_length = 2046
def construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id):

    text_ids = tokenizer.encode(text, truncation = True, add_special_tokens=False, max_length = max_length)
    # construct input token ids
    input_ids = [cls_token_id] + text_ids + [sep_token_id]
    # construct reference token ids 
    ref_input_ids = [cls_token_id] + [ref_token_id] * len(text_ids) + [sep_token_id]

    return torch.tensor([input_ids], device=device), torch.tensor([ref_input_ids], device=device), len(text_ids)

def construct_input_ref_pos_id_pair(input_ids):
    seq_length = input_ids.size(1)
    position_ids = torch.arange(seq_length, dtype=torch.long, device=device)
    # we could potentially also use random permutation with `torch.randperm(seq_length, device=device)`
    ref_position_ids = torch.zeros(seq_length, dtype=torch.long, device=device)

    position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
    ref_position_ids = ref_position_ids.unsqueeze(0).expand_as(input_ids)
    return position_ids, ref_position_ids
    
def construct_attention_mask(input_ids):
    return torch.ones_like(input_ids)

Import dataset and take a few examples from it for testing purposes

Here we import the papers dataset

In [11]:
from datasets import load_dataset
import numpy as np
cogs402_ds = load_dataset("danielhou13/cogs402dataset")["test"]

Using custom data configuration danielhou13--cogs402dataset-144b958ac1a53abb
Reusing dataset parquet (/root/.cache/huggingface/datasets/danielhou13___parquet/danielhou13--cogs402dataset-144b958ac1a53abb/0.0.0/7328ef7ee03eaf3f86ae40594d46a1cec86161704e02dd19f232d81eee72ade8)


  0%|          | 0/2 [00:00<?, ?it/s]

Here we import the news dataset

In [9]:
# cogs402_ds2 = load_dataset('hyperpartisan_news_detection', 'bypublisher')['validation']
# val_size = 5000
# val_indices = np.random.randint(0, len(cogs402_ds2), val_size)
# val_ds = cogs402_ds2.select(val_indices)
# labels2 = map(int, val_ds['hyperpartisan'])
# labels2 = list(labels2)
# val_ds = val_ds.add_column("labels", labels2)

In [12]:
#set 1 if we are dealing with a positive class, and 0 if dealing with negative class
def custom_forward(inputs, position_ids=None, attention_mask=None):
    preds = predict(inputs,
                   position_ids=position_ids,
                   attention_mask=attention_mask
                   )
    return torch.softmax(preds, dim = 1)

Perform Layer Integrated Gradients using the longformer's embeddings

In [13]:
def summarize_attributions(attributions):
    attributions = attributions.sum(dim=-1).squeeze(0)
    attributions = attributions / torch.linalg.norm(attributions)
    return attributions

In [14]:
lig = LayerIntegratedGradients(custom_forward, model.longformer.embeddings)

In [15]:
vis_data_records = []
all_attributions = {}
all_tokens = {}
def get_token_attributions(dataset, example):
  text = cogs402_ds['text'][example]
  label = cogs402_ds['labels'][example]

  input_ids, ref_input_ids, sep_id = construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id)
  position_ids, ref_position_ids = construct_input_ref_pos_id_pair(input_ids)
  attention_mask = construct_attention_mask(input_ids)

  indices = input_ids[0].detach().tolist()
  all_tokens = tokenizer.convert_ids_to_tokens(indices)

  all_attributions[str(example)] = all_tokens

  attributions, delta = lig.attribute(inputs=input_ids,
                                    baselines=ref_input_ids,
                                    return_convergence_delta=True,
                                    additional_forward_args=(position_ids, attention_mask),
                                    target=1,
                                    n_steps=50,
                                    internal_batch_size = 2)

  attributions_sum = summarize_attributions(attributions)

  all_attributions[str(example)] = attributions_sum

  score = predict(input_ids, position_ids, attention_mask)

  # storing couple samples in an array for visualization purposes
  vis_data_records.append(viz.VisualizationDataRecord(
                        attributions_sum,
                        torch.softmax(score, dim = 1).max(),
                        torch.argmax(torch.softmax(score, dim = 1)),
                        label,
                        str(1),
                        attributions_sum.sum(),       
                        all_tokens,
                        delta)
  )

In [16]:
get_token_attributions(cogs402_ds, 976)
get_token_attributions(cogs402_ds, 891)
get_token_attributions(cogs402_ds, 605)
get_token_attributions(cogs402_ds, 148)

RuntimeError: ignored

Visualize the attributions for the words

In [17]:
# # storing couple samples in an array for visualization purposes
# score_vis = viz.VisualizationDataRecord(
#                         attributions_sum,
#                         torch.softmax(score, dim = 1).max(),
#                         torch.argmax(torch.softmax(score, dim = 1)),
#                         label,
#                         str(1),
#                         attributions_sum.sum(),       
#                         all_tokens,
#                         delta)

print('\033[1m', 'Visualization For Score', '\033[0m')
viz.visualize_text(vis_data_records)

 Visualization For Score 


Next we might want to look in-depth about the attribution scores for each token of an example

In [16]:
example = 976
attributions_sum = all_attributions[str(example)]

See which words had the strongest (most positive and most negative) attributions. Change the number of tokens you wish to visualize for your needs

In [17]:
def get_topk_attributed_tokens(attrs, k=20):
    values, indices = torch.topk(attrs, k)
    top_tokens = [all_tokens[idx] for idx in indices]
    return top_tokens, values, indices

In [18]:
def get_botk_attributed_tokens(attrs, k=20):
    values, indices = torch.topk(attrs, k, largest=False)
    top_tokens = [all_tokens[idx] for idx in indices]
    return top_tokens, values, indices

Convert the values, index of the values, and the token into a pandas Dataframe for visualization. It will be sorted by highest value for attributions to lowest. Alternatively, if youre looking for the lowest attributions, it goes from lowest to highest.

In [19]:
top_words_start, top_words_val_start, top_word_ind_start = get_topk_attributed_tokens(attributions_sum)
bot_words_start, bot_words_val_start, bot_word_ind_start = get_botk_attributed_tokens(attributions_sum)

df_high = pd.DataFrame({'Word(Index), Attribution': ["{} ({}), {}".format(word, pos, round(val.item(),2)) for word, pos, val in zip(top_words_start, top_word_ind_start, top_words_val_start)]})

df_low = pd.DataFrame({'Word(Index), Attribution': ["{} ({}), {}".format(word, pos, round(val.item(),2)) for word, pos, val in zip(bot_words_start, bot_word_ind_start, bot_words_val_start)]})
# df_start.style.apply(['cell_ids: False'])

# ['{}({})'.format(token, str(i)) for i, token in enumerate(all_tokens)]

NameError: ignored

In [35]:
df_high

,"Word(Index), Attribution"
0,"Ġtraining (1544), 0.49"
1,"Ġtraining (1687), 0.46"
2,"Ġtraining (1700), 0.38"
3,"Ġtraining (1593), 0.28"
4,"Ġtraining (1659), 0.27"
5,"Ġtraining (1705), 0.25"
6,"Ġtraining (1493), 0.16"
7,"Ġtraining (1791), 0.15"
8,"Ġtraining (1506), 0.13"
9,". (1688), 0.1"


In [36]:
df_low

,"Word(Index), Attribution"
0,". (251), -0.18"
1,"ĠComputer (10), -0.1"
2,"ĠInstead (252), -0.06"
3,"Ġlinguistic (1579), -0.05"
4,"Ġlanguage (1571), -0.05"
5,"Ġof (9), -0.04"
6,"ĠLanguage (21), -0.04"
7,"Ġword (1625), -0.03"
8,"Ġwords (1643), -0.02"
9,"ĠVision (11), -0.02"


In [ ]:
d = {"tokens":all_tokens, "attribution":attributions_sum[:len(all_tokens)].cpu()}

We notice that there are many repeating tokens in each example that have different positions. While we might want to know how the position plays into the attributions, if we want to know strictly based on tokens, we can add all the duplicate tokens together to get the aggregate attribution for each token.

In [ ]:
df_attrib = pd.DataFrame(d)
aggregation_functions = {'attribution': 'sum'}
df_new = df_attrib.groupby(df_attrib['tokens']).aggregate(aggregation_functions)

In [ ]:
highest_attrib_tokens = df_new.sort_values(by=['attribution'], ascending=False)
highest_attrib_tokens[:10]

,attribution
tokens,
Ġtraining,2.667580
.,0.623735
",",0.294686
Ġhuman,0.259636
Ġbias,0.178366
Ġmodel,0.170915
Ġlearning,0.158571
Ġa,0.157962
Ġto,0.148793


In [ ]:
lowest_attrib_tokens = df_new.sort_values(by=['attribution'])
lowest_attrib_tokens[:10]

,attribution
tokens,
Ġlanguage,-0.088873
Ġwords,-0.063263
ĠComputer,-0.054022
ĠLanguage,-0.047863
Ġlinguistic,-0.032388
Ġword,-0.029338
Ġsentences,-0.028764
Ġ,-0.025477
ĠInstead,-0.023174


We can also find the aggregate total for each token over the entire dataset in order to find which words have the highest and lowest attribution when the model is predicting positive and when the model is predicting negative. 

In [ ]:
print(len(all_tokens), attributions_sum[:len(all_tokens)].shape)

2048 torch.Size([2048])


In [ ]:
from tqdm import tqdm
aggregate_attrib_zero = []
aggregate_attrib_ones = []
aggregation_function = {'attribution': 'sum'}

for i in tqdm(range(len(cogs402_ds))):
  text = cogs402_ds[i]['text']
  label = cogs402_ds[i]['labels']
  input_ids, ref_input_ids, sep_id = construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id)
  position_ids, ref_position_ids = construct_input_ref_pos_id_pair(input_ids)
  attention_mask = construct_attention_mask(input_ids)

  indices = input_ids[0].detach().tolist()
  all_tokens = tokenizer.convert_ids_to_tokens(indices)

  attributions = lig.attribute(inputs=input_ids,
                                    baselines=ref_input_ids,
                                    additional_forward_args=(position_ids, attention_mask),
                                    target=label,
                                    n_steps=20,
                                    internal_batch_size = 2)
  
  attributions_sum = summarize_attributions(attributions)
  
  d = {"tokens":all_tokens, "attribution":attributions_sum[:len(all_tokens)].cpu()}  
  df_attrib = pd.DataFrame(d)
  df_attrib = df_attrib.groupby(df_attrib['tokens']).aggregate(aggregation_function)

  if label == 0:
    aggregate_attrib_zero.append(df_attrib)
  else:
    aggregate_attrib_ones.append(df_attrib)

100%|██████████| 1070/1070 [3:07:03<00:00, 10.49s/it]


Here we get the attributions for the negative class, meaning that these words are the key words that help the model predict the negative class.

In [ ]:
df_attrib_zero = pd.concat(aggregate_attrib_zero)
df_attrib_zero = df_attrib_zero.reset_index(level=0)
df_attrib_zero = df_attrib_zero.groupby(df_attrib_zero['tokens']).aggregate(aggregation_function)
df_attrib_zero['attribution'] = df_attrib_zero['attribution'].div(len(aggregate_attrib_zero))
highest_attrib_tokens_all = df_attrib_zero.sort_values(by=['attribution'], ascending=False)
highest_attrib_tokens_all[:10]

,attribution
tokens,
Ġprogramming,0.221216
Ġprogram,0.147403
Ġprograms,0.140019
Ġlanguages,0.131498
Ġlanguage,0.094632
Ġcode,0.081231
Ġa,0.072384
Ġ.,0.064827
Ġsoftware,0.060958


Here we get the highest attributions for the positive class, meaning that these are the key words that help the model predict the positive class.

In [ ]:
df_attrib_ones = pd.concat(aggregate_attrib_ones)
df_attrib_ones = df_attrib_ones.reset_index(level=0)
df_attrib_ones = df_attrib_ones.groupby(df_attrib_ones['tokens']).aggregate(aggregation_function)
df_attrib_ones['attribution'] = df_attrib_ones['attribution'].div(len(aggregate_attrib_ones))
highest_attrib_tokens_all2 = df_attrib_ones.sort_values(by=['attribution'], ascending=False)
highest_attrib_tokens_all2[:10]

,attribution
tokens,
.,0.504835
Ġlearning,0.287378
",",0.238612
Ġneural,0.231642
Ġdata,0.142503
Ġthe,0.137999
Ġto,0.119790
Ġtraining,0.101769
Ġof,0.089441


Save the pandas dataframe into a csv to access it in the future without having to run through the entire dataset

In [ ]:
highest_attrib_tokens_all.to_csv('longformer_emb_zeros_papers.csv')  
highest_attrib_tokens_all2.to_csv('longformer_emb_ones_papers.csv')  

In [ ]:
highest_attrib_tokens_all.to_csv('/content/drive/MyDrive/cogs402longformer/longformer_emb_zeros_papers.csv')  
highest_attrib_tokens_all2.to_csv('/content/drive/MyDrive/cogs402longformer/longformer_emb_ones_papers.csv')  